In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 56.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 83.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=6682b33253dcc62b96ebbb6bebc32201988b29d05aecbd64460a33be55e37d09
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal-5.0.1:
      Successfully uninstalled tzlocal-5.0.1


# Libraries & Functions

In [3]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

In [4]:
from simpletransformers.classification import ClassificationModel
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

def calculate_scores(y_test, y_pred, average = "binary"):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average = average)
    recall = recall_score(y_test, y_pred, average = average)
    f1 = f1_score(y_test, y_pred, average = average)
    return [accuracy, precision, recall, f1]

In [8]:
model_names = ["distilbert", "distilbert", "bert", "bert"]
checkpoint_names = ["distilbert-base-uncased", "distilbert-base-multilingual-cased", "dccuchile/bert-base-spanish-wwm-uncased", "dbmdz/bert-base-german-uncased"]

In [9]:
focus_names = ["Growth Form", "Life Form"]
focus_codes = ["1.2.1", "2.3.1"]

# Input Data

In [22]:
raw_datasets = dict()

In [23]:
def fix_WIKI(name, description):
    for n in name.split(" "):
        description = str(description).replace(n.lower(), "")
    return description.strip()

## Spanish Wikipedia - WIKI_GIFT_ESP Dataset

In [30]:
working_dir = "..//input//wiki-gift-esp-final//" 

df_WIKI_ESP_Cat =  pd.read_excel(working_dir + "WIKI_GIFT_ESP.xlsx")
df_WIKI_ESP_Cat_Preproc = df_WIKI_ESP_Cat.drop_duplicates(subset = ["BERT_description"])
df_WIKI_ESP_Cat_Preproc["BERT_description"] = df_WIKI_ESP_Cat_Preproc[["name", "BERT_description"]].apply(lambda x: fix_WIKI(x[0], x[1]), axis = 1)
df_WIKI_ESP_Cat_Preproc = df_WIKI_ESP_Cat_Preproc[df_WIKI_ESP_Cat_Preproc["BERT_description"].apply(lambda x: len(str(x).split(" ")))>10]
raw_datasets["WIKI_ESP"] = df_WIKI_ESP_Cat_Preproc

/tmp/ipykernel_29/1635896773.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_WIKI_ESP_Cat_Preproc["BERT_description"] = df_WIKI_ESP_Cat_Preproc[["name", "BERT_description"]].apply(lambda x: fix_WIKI(x[0], x[1]), axis = 1)


## German Wikipedia - WIKI_GIFT_DE Dataset

In [31]:
working_dir = "..//input//wiki-gift-de-final//" 

df_WIKI_DE_Cat =  pd.read_excel(working_dir + "WIKI_GIFT_DE.xlsx")
df_WIKI_DE_Cat_Preproc = df_WIKI_DE_Cat.drop_duplicates(subset = ["BERT_description"])
df_WIKI_DE_Cat_Preproc["BERT_description"] = df_WIKI_DE_Cat_Preproc[["name", "BERT_description"]].apply(lambda x: fix_WIKI(x[0], x[1]), axis = 1)
df_WIKI_DE_Cat_Preproc = df_WIKI_DE_Cat_Preproc[df_WIKI_DE_Cat_Preproc["BERT_description"].apply(lambda x: len(str(x).split(" ")))>10]
raw_datasets["WIKI_DE"] = df_WIKI_DE_Cat_Preproc

/tmp/ipykernel_29/574063204.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_WIKI_DE_Cat_Preproc["BERT_description"] = df_WIKI_DE_Cat_Preproc[["name", "BERT_description"]].apply(lambda x: fix_WIKI(x[0], x[1]), axis = 1)


## Preprocess Datasets

In [32]:
label_map = {
    "Growth Form": {"herb": 0, "shrub": 1, "tree": 2},
    "Life Form": {"phanerophyte": 0, "chamaephyte": 1, "hemicryptophyte": 2, "cryptophyte": 3, "therophyte": 4},
}


In [33]:
preprocessed_dataset_dict = {}
for focus_name, focus_code in zip(focus_names, focus_codes):
    for dataset_name in list(raw_datasets.keys()):
        labelencoder = LabelEncoder()

        dataset_masked = raw_datasets[dataset_name][raw_datasets[dataset_name][focus_code].notna()]
        dataset_masked = dataset_masked[dataset_masked[focus_code].apply(lambda x: x in label_map[focus_name].keys())]
        dataset_masked[focus_code + "_encoded"] = labelencoder.fit_transform(dataset_masked[focus_code])

        indices_train, indices_test \
            = train_test_split(dataset_masked.index.values, test_size=0.25, random_state=42)
            
        df_train = dataset_masked.loc[indices_train, ["BERT_description", focus_code + "_encoded"]]
        df_train.columns = ["text", "labels"]
        df_test = dataset_masked.loc[indices_test, ["BERT_description", focus_code + "_encoded"]]
        df_test.columns = ["text", "labels"]
        
        preprocessed_dataset_dict[dataset_name, focus_name, "train"] = df_train
        preprocessed_dataset_dict[dataset_name, focus_name, "validation"] = df_test

# Model Training & Evaluation

In [34]:
if torch.cuda.is_available():  # Tell PyTorch to use the GPU. 
    device = torch.device("cuda") 
    print('There are %d GPU(s) available.' % torch.cuda.device_count()) 
    print('We will use the GPU:', torch.cuda.get_device_name(0)) # If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    
import gc
torch.cuda.empty_cache()
gc.collect()

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


4796

In [36]:
results_list = []

for model_name, model_checkpoint in zip(model_names[:], checkpoint_names[:]):
    for focus_name, focus_code in zip(focus_names, focus_codes):
        for dataset_name in list(raw_datasets.keys())[:]:

            print(model_name, focus_name, dataset_name)

            model = ClassificationModel(
                model_name,
                model_checkpoint,
                num_labels = preprocessed_dataset_dict[dataset_name, focus_name, "train"]["labels"].nunique(),
                args = {"num_train_epochs": 3, "train_batch_size":8, "eval_batch_size":8, "reprocess_input_data": True, "overwrite_output_dir": True, "save_model_every_epoch": False, "save_eval_checkpoints": False, "max_seq_length": 512}, #"weight_decay": 0.01, "learning_rate": 2e-5, 
            )
            # Train the model
            model.train_model(preprocessed_dataset_dict[dataset_name, focus_name, "train"])

            # Evaluate the model
            result, model_outputs, wrong_predictions = model.eval_model(preprocessed_dataset_dict[dataset_name, focus_name, "validation"])
            preprocessed_dataset_dict[dataset_name, focus_name, "validation"]["prediction"] = np.argmax(model_outputs, axis=1)
            results = calculate_scores(preprocessed_dataset_dict[dataset_name, focus_name, "validation"]["labels"], preprocessed_dataset_dict[dataset_name, focus_name, "validation"]["prediction"], average = "macro")
            results_list.append([dataset_name, focus_name] + results + [model_checkpoint])

            torch.cuda.empty_cache()
            gc.collect()
            print(results)

df_results = pd.DataFrame(results_list, columns=["Dataset", "Trait", "Accuracy", "Precision", "Recall", "F1-Score", "Model"])

distilbert Growth Form WIKI_ESP


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2930 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/367 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/160 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/426 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/54 [00:00<?, ?it/s]

[0.8685446009389671, 0.8634695780634425, 0.8052200621706891, 0.8221444317849576]


In [37]:
df_results

,Dataset,Trait,Accuracy,Precision,Recall,F1-Score,Model
0,WIKI_ESP,Growth Form,0.850563,0.820488,0.807822,0.813639,distilbert-base-uncased
1,WIKI_DE,Growth Form,0.863558,0.820741,0.827731,0.823429,distilbert-base-uncased
2,WIKI_ESP,Life Form,0.682594,0.608215,0.586844,0.593576,distilbert-base-uncased
3,WIKI_DE,Life Form,0.673709,0.723202,0.561372,0.522810,distilbert-base-uncased
4,WIKI_ESP,Growth Form,0.850563,0.819941,0.807210,0.813001,distilbert-base-multilingual-cased
5,WIKI_DE,Growth Form,0.894646,0.863231,0.843062,0.852458,distilbert-base-multilingual-cased
6,WIKI_ESP,Life Form,0.537543,0.371479,0.422081,0.375927,distilbert-base-multilingual-cased
7,WIKI_DE,Life Form,0.605634,0.504986,0.473973,0.473187,distilbert-base-multilingual-cased
8,WIKI_ESP,Growth Form,0.853634,0.826186,0.810363,0.817524,dccuchile/bert-base-spanish-wwm-uncased
9,WIKI_DE,Growth Form,0.873921,0.839796,0.811019,0.823940,dccuchile/bert-base-spanish-wwm-uncased


In [41]:
df_results.to_excel("outputs//NonEnglishCategoricalTraitClassification_Encoder_Results.xlsx", index = False)

In [39]:
!ls

NonEnglishCategoricalTraitClassification_Encoder_Results.xlsx  outputs
cache_dir						       runs
